### Install PYTHON dependencies

In [1]:
pip install fhirpy Authlib python-dotenv

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import datetime
import os
import uuid

import requests
from authlib.jose import jwt
from dotenv import load_dotenv

from fhirpy import SyncFHIRClient

### load environment variables

`cp example/.env.tpl example/.env`   
`CLIENT_ID` - your epic app Non-Production Client ID   
`PRIVATE_KEY_PATH` - path to your private .pem file (see [documentation](https://fhir.epic.com/Documentation?docId=oauth2&section=Creating-Key-Pair))

In [3]:
load_dotenv("./example/.env")

True

### Generate JWT and get access token

In [4]:
BASE_URL = "https://fhir.epic.com/interconnect-fhir-oauth"

In [5]:
jwt_header = {"alg": "RS384", "typ": "JWT"}

In [6]:
# Token expiration time should not be more then 5 minutes since the current
exp = int((datetime.datetime.now() + datetime.timedelta(minutes=4)).timestamp())

payload = {
    "iss": os.getenv("CLIENT_ID"),
    "sub": os.getenv("CLIENT_ID"),
    "aud": f"{BASE_URL}/oauth2/token",
    "jti": str(uuid.uuid4()),
    "exp": exp,
}

In [7]:
with open(os.getenv("PRIVATE_KEY_PATH")) as f:
    private_key = f.read()

In [8]:
encoded_token = jwt.encode(jwt_header, payload, private_key)

In [9]:
headers = requests.structures.CaseInsensitiveDict(
    [("Content-Type", "application/x-www-form-urlencoded")]
)

In [10]:
data = {
    "grant_type": "client_credentials",
    "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
    "client_assertion": encoded_token
}

In [11]:
response = requests.post(
    f"{BASE_URL}/oauth2/token",
    headers=headers,
    data=data,
)

In [12]:
access_token = response.json()["access_token"]

### Init FHIR Client

In [13]:
FHIR_BASE_URL = f"{BASE_URL}/api/FHIR/R4"

In [14]:
client = SyncFHIRClient(
    FHIR_BASE_URL,
    authorization=f"Bearer {access_token}",
)

### Get patient

In [ ]:
client.reference("Patient", "eJzlzKe3KPzAV5TtkxmNivQ3").to_resource().serialize()